# Tracking Votes with Redis Strings

This chapter will took at using the Redis string datatype to track user votes on a
website. User voting can take many forms: up/down votes, thumbs up, likes, but essentially
all of these experiences distill down to recording positive and negative impressions
from the user in the database.

This chapter will look at using the Redis string datatype to maintain a global count of
votes on a particular blog post. To developers unfamiliar with Redis, maintaining a vote
count using a string might sound incorrect, but the Redis String type is different from
the string types in other programming languages you are familiar with.

In this chapter, we will learn about:

* Storing data in Redis strings
* Manipulating data in Redis strings
* How Redis strings differ from string datatypes 
* Working with bitmaps, integers and floating points

At the end of the chapter, you should understand how to use the Redis string datatype to
implement key-value operations and apply Redis string operations to a variety of development problems.

## Strings

Redis provides a string datatype that is used to associate data with a particular key.
Redis strings are the most basic datatype available in Redis and one of the first datatypes
that users learn to work with.  Our Hello Work program in the chapter on connecting to Redis used the string datatype.

Most programmers associate string datatypes with text data like the string datatype in
most modern programming languages.  Redis strings are more like buffers that can store a wide range of data.  Redis strings can be used to store:

* Integers
* Floating point numbers
* Bitmaps
* Strings
* Binary objects

String values in Redis can be up to 512 MB in size, allowing you to store substantial binary objects including images, sound files, and object files.

* * *

> **Note**
>
> Redis keys are also binary safe, so although most programming examples show Redis keys
> as strings, you can use any binary sequence as a key in Redis.  Keys can also be up
> 512 MB in length, but we caution against using very long keys as they can have an
> adverse effect on performance.  Standard practice in Redis is to use a structured key
> names with a schema, such as `comment:1234:reply.to` or `comment:1234:reply-to`.
>
> We also recommend that you do not use very short names for Redis keys either, ease of 
> use and clarity needs to be balanced out against performance.  The performance 
> improvements of a key like `c1234r2` versus `comment:1234:reply.to` are minimal and
> don't outweigh the loss of clarity.
>

* * *

The current version of Redis (3.2) provides 24 different commands to work with the string
data type.  Redis provides commands to:

* Read and write string values
* Manipulate strings as integers
* Manipulate strings as floating point numbers
* Use strings to store bitmaps

Redis allows you several ways to perform calculations on your data *without* having to read
the data into your application, manipulate it, and then write it back to the server.

## Tracking Votes

In our examples, votes will be integer values, both positive and negative, associated with
a particular blog post. Our system will track raw votes, each user can either vote up or
down for a given post. The resulting vote count is displayed to users as a way of
prioritizing posts to read.

## Data Storage Conventions

Our votes will be stored in Redis using the string datatype. One string will be stored for
each item under the key `item:{item_no}:vote-count` and the string will be used to store
an integer vote count. An up-vote (positive vote) is tracked as a `+1` and a down-vote
(negative vote) is tracked as a `-1`. The votes for a particular item are the sum of all
the up and down votes.

To improve the clarity of our code, the Notebook environment includes a few utility
functions, allowing you to focus on the Redis implementation details. In this chapter, the
main functions that will be used from the `workshop` package are:

* `item_vote_key` - generates the proper key from an item id
* `create_item_vote` - clears current vote totals from the database
* `get_item_vote` - returns the vote total for an item 
* `users_vote_key` - returns the key for tracking a users vote
* `clear_users_vote` - clears a users voting record

## Vote Counting

### Counting an Upvote

When a user views a blog post on our website, they are given the opportunity to vote on the
post.  If the user thought the post was a good post and want to recommend the post to
others, they can upvote the post.  In our first version of counting upvotes, we aren't going to worry about duplicates, so we need code that will:

* Generate an appropriate Redis key
* Increment item vote count

This can be accomplished with a single Redis command - INCR.

The INCR (**INCR**ement) command operates on an integer value stored in Redis as a string
and increments the value of the key on the server by one.

*Run the example below by selecting the code cell and pressing SHIFT + ENTER*

In [ ]:
def upvote_item(r, item_id):
    "Upvotes an item and stores it in the Redis database"
    
    # get our key
    key = workshop.strings.item_vote_key(item_id)
 
    return r.incr(key)


# reset our database state
workshop.strings.clear_item_vote(r, 20)

# execute our example
print 'Vote count for item 20: {}'.format(upvote_item(r, 20))


After running the sample code, you should see the result `Vote count for item 20: 1` output
below the cell.

Notice in the code, we never create the key `item:20:vote-count` or check that it exists.
Our code always performs an increment operation. We can write our code this way because
Redis is designed to provide sensible defaults whenever data is missing.  When INCR is called with a key that doesn't exist, Redis assumes a default value of 0. increments it and stores the result.  

It is also important to point out that in the proceeding example, we never read the 
current value of the key into memory.  The increment command operates on the value
stored in the database.  This improves performance for our code and makes it
easier to support concurrent updates since we do not require a transaction.

* * *

> **Tip**
>
> While Redis is designed to handle missing data with sensible defaults, not all
> erroneous operations are handled quietly.  If the INCR command is applied to a 
> string value that can't be represented as an *integer* (including floating point 
> numbers), Redis will return an error.
>

* * *


As mentioned previously, our Notebook is intended to be an interactive environment that
you can experiment with. Take some time to experiment with the example code to get more 
familiar with the Notebook environment.  Some possible experiments to try:

* Add a Python error to the code 
* Add a Redis error to the code
* Use the GET and SET commands from the Hello World program

### Counting a Downvote

The code an process for handling downvotes is very similar to the code for handling upvotes.  At the user experience level, a visitor to our site can also downvote posts they do not find appealing or believe aren't worth the time for other readers to read.  Like 
before, in our first version of downvoting, we aren't going to worry about a user
casting duplicate votes.  The structure for the downvoting code is nearly the same as 
the upvoting code: 

* Generate an appropriate Redis key
* Decrement item vote count

This can also be accomplished with a single Redis command - DECR.

The DECR (**DECR*ement) command command operates on an integer value stored as a string and decrements the value by one. 

*Select the code cell below and press SHIFT + ENTER to run the downvote example.*

In [ ]:
def downvote_item(r, item_id):
    "Downvotes an item and stores it in the Redis database"
    
    # get our key
    key = workshop.strings.item_vote_key(item_id)
 
    return r.decr(key)


# reset our database state
workshop.strings.clear_item_vote(r, 20)

# execute our example
print "Vote count for item 20: {}".format(downvote_item(r, 20))


After running the sample code, you should see the message `Vote count for item 20: -1`
printed under the cell.

As mentioned previously, this example is nearly identical to the `upvote_item` example.
The only different between the two examples is the Redis command used: INCR for up votes
and DECR for down votes. All of points we discussed about INCR: sensible defaults,
on-server operation, and transactional also apply to the DECR command.

### Weighted Votes

Some voting systems prefer to give different weights to upvotes and downvotes. Often a
voting system will be biased towards positive votes and weight them higher than negative
votes as a simple way to discourage unwanted social behaviors. If we decide to change our
voting system to give a higher weight to upvotes than to downvote, a simple refactoring 
to our upvote/downvote routines will implement the change.

In the following example, we modify our initial
`upvote_item` and `downvote_item` to give upvotes twice the weight as downvotes.  We refactor the code to use the INCRBY command instead of the we need to change the INCR command.  The INCRBY (**INCR**ease **BY**) command takes a key and a step or increment
parameter and updates the value in the database by the supplied increment.  In our
case we increment each upvote by two establishing a two to one weighting of positive to negative votes.

*Select the code cell below and press SHIFT + ENTER to execute the example.*

In [ ]:
def upvote_item(r, item_id):
    "Upvotes an item and stores it in the Redis database"
    
    # get our key
    key = workshop.strings.item_vote_key(item_id)
 
    return r.incrby(key, 2)

def downvote_item(r, item_id):
    "Downvotes an item and stores it in the Redis database"
    
    # get our key
    key = workshop.strings.item_vote_key(item_id)
 
    return r.decr(key)

# reset our database state
workshop.strings.clear_item_vote(r, 20)

# simulate a vote sequence
upvote_item(r, 20)
upvote_item(r, 20)
upvote_item(r, 20)
downvote_item(r, 20)

# see the vote results
print "Vote count for item 20: {}".format(workshop.strings.get_item_vote(r, 20))


The `get_item_vote` method will return a vote count of `5` after running the sample.

* * *

> **Tip**
>
> The *redis-py* library allows you to pass an optional step argument to the `incr` 
> and `decr` functions, which get mapped to the INCRBY and DECRBY commands.
>

* * *


### Tracking Votes

Now that we've seen a couple examples of vote recording functions, we need to flesh out or
implementation to demonstrate a more realistic approach to the problem. Most websites
don't allow users to vote more than once on a given item. In this iteration of the code,
we will update our upvote and downvote methods to check that a user has not already
registered a vote on a particular item. A user's vote will only be applied once, otherwise
the function will ignore the vote and return `False`.

We will be using the Redis bitmap "type" to maintain a record of votes. As we saw earlier,
Redis has commands that treat strings as integers and performs basic arithmetic on the
values. Redis also has commands that treat strings as bitmaps and can perform standard bit
operations: test, set, and logical. Redis represents bitmaps as a variable width vector of
bits with a zero-based indexing scheme. Bitmaps can grow up to Redis' maximum value size
(512 MB).

We can augment our original functions with two additional commands, GETBIT and SETBIT, to
record votes using Redis' bitmaps. In our code, we will store the users votes in the key
`user:{user_id}:votes` with a bitmap indexed by the item id. A zero value in the bitmap
will indicate no vote and a one value in the bitmap will indicate the user voted.

To prevent the user from having a duplicate vote we need to:

* Lookup the users vote for a particular item_id
* Check for a recorded vote
* Record that the user voted
* Apply the value of the users vote

The example code below shows a modified version of our original voting functions with
duplicate checks added.  *Please run the example by selecting the code cell and pressing SHIFT + ENTER to execute the code.*

In [ ]:
def user_voted(r, user_id, item_id):
    "Checks if a user has already voted on an item"
    key = workshop.strings.users_vote_key(user_id)
    voted = r.getbit(key, item_id)
    
    # voted is returned as a bit (0/1), cast to Python boolean
    return bool(voted)
    
    
def upvote_item(r, user_id, item_id):
    "Processes a user's upvote and stores it in Redis (with duplicate checking)"
    
    if not user_voted(r, user_id, item_id):
        r.setbit(workshop.strings.users_vote_key(user_id), item_id, 1)
        r.incrby(workshop.strings.item_vote_key(item_id), 2)
        return True
    else:
        return False
        

def downvote_item(r, user_id, item_id):
    "Processes a user's downvote and stores it in Redis (with duplicate checking)"
    
    if not user_voted(r, user_id, item_id):
        r.setbit(workshop.strings.users_vote_key(user_id), item_id, 1)
        r.decr(workshop.strings.item_vote_key(item_id))
        return True
    else:
        return False

workshop.strings.clear_item_vote(r, 20)
workshop.strings.clear_item_vote(r, 21)
workshop.strings.clear_users_vote(r, 3001)
workshop.strings.clear_users_vote(r, 3002)
    

upvote_item(r, 3001, 20)
upvote_item(r, 3001, 21)
upvote_item(r, 3001, 20)

downvote_item(r, 3002, 20)
downvote_item(r, 3002, 20)

print "Vote count for item 20: {}".format(workshop.strings.get_item_vote(r, 20))
print "Vote count for item 21: {}".format(workshop.strings.get_item_vote(r, 21))
print "Vote count for item 22: {}".format(workshop.strings.get_item_vote(r, 22))


The result from our simulated vote sequences should be:

item | vote
-----|-----
20 | 1
21 | 2
22 | 0

* * *

> **Note**
>
> One thing that you might notice in the implementation of our new voting functions is that
> there is no explicit transactions or currency control applied.  In this chapter, we want
> to focus on basic String operations and currency control will be discussed later.
> 
> However, this is a good point to discuss how our Python method invocations are 
> translated into Redis operations on the database server.  Each method call is executed
> as a single call to and response from the Redis database.  In the above example, the
> call to `getbit` in the `upvote_item` is executed in a separate call/response from
> the `setbit` method in the `upvote_item` and`downvote_item` functions.  Since these
> commands are executed in a separate call and response, our code suffers from the
> classic concurrent update problem.  As written, there is nothing in the code to 
> prevent another execution thread from modifying the data between method invocations
> in our function.  
> 
> The chapter on transactions and concurrency control will show you the features of 
> Redis that you can use to maintain proper update consistency.  For now, concentrate
> on learning the basic Redis operations and don't be concerned with concurrency.
>

* * *


Experiment with the implementation of upvoting and downvoting.  Go back and update the implementation of upvoting and downvoting allowing the user to change their vote after the vote is cast.

## Counting with Floating Point Numbers

In addition to the ability to treat strings as integers, bitmaps, and text, Redis can
treat strings as floating point numbers. Redis provides the INCRBYFLOAT (**INCR**ement
**BY** **FLOAT**) command which takes a key and an increment, which can be negative, and
updates the value of the key in the database by the increment specified.

* * *

> **Tip**
> 
> Redis does not provide a subtype for floats or any of the other types of data that can
> be represented using the String datatype. The type operation returns the type `string`
> when called on any of the subtypes. At the same time, the regular `get` and `set` string > operations can also be applied to any of the subtypes. 
>
> When you call the `get` operation on a bitmap, Redis returns an escaped string 
> representing the bitmap. On an integer or a floating point, you get the string 
> representation of the number. You can even use the textual representation of a number
> or bitmap (use `0x00` escapes) to set a value in Redis. 
>
> All of the string operations can be applied to any subtype, but not all combinations
> work.
>
> *Optional Exercise*: Try using the various string commands on different subtypes
> (bitmaps, integers, floats, text) and see how the various combinations interact.
>
* * *

In [ ]:

# Use this cell to experiment with string commands


## Expanding on Get and Set

We introduced the basic Redis String manipulation commands GET and SET in our first Hello
World program. GET and SET have some important variants to learn about which allow the
user to operate multiple keys, control the time-to-live of keys and perform basic
test-and-set operations.

### Working with Multiple Keys

The Redis GET AND SET command have variants that can operate on multiple keys at a time.
The GET command is paired with the MGET (**M**ultiple **GET**) command that takes a
sequence of keys and returns the value of those keys. The MSET (**M**ultiple **SET**)
command works the same way. It takes a sequence of key value pairs and sets the values of
all those keys.

* * * 

> **Tip**
> 
> The multiple key operations are important because they allow developers to operate on a
> consistent snapshot of a set keys.  As we mentioned earlier, each method call in our 
> program is executed as a single call/response to the server.  If we make multiple GET or
> SET calls, concurrent modifications could change the data in-between calls.
>
 
* * *


### Expiring Keys

Keys written to Redis are stored until deleted provided persistence is enabled. Without
persistence, keys are only stored until the Redis server is shutdown or rebooted. In some
applications, cached values are only valid for a limited time period. Redis allows you to
optionally provide an expiration time when you set data.

With the SET command, Redis allows you to optionally specify the expiration time of a key
using the EX parameter for seconds or the PX parameter for milliseconds. The *redis-py*
library allows you to specify both of these optional parameters as keyword arguments to
the SET function.

Redis provides two other (redundant) functions for setting keys with an expiration time.
The PSETEX can be used to set a key with a millisecond expiration time and the SETEX
command that allows you to set the expiration in seconds. Since these functions are
redundant to the SET command options, it is possible they may be deprecated and removed in
future versions of Redis.

### Test and Set Operations

Redis provides a third variant of the SET operation to perform "test-and-set" operations.
The Redis SETNX (**SET** **N**ot e**X**ists) command allows you to provisionally set the
value of a key, provided that the key is not already defined. Redis returns a result of
`1` if the key was set and `0` if the key was not set.

The SET command provides optional `NX` and `XX` flags that update the key only when it
doesn't already exist (`NX`) and only when it already exists (`XX`). These flags can be
used in combination with the expiration parameters (`PX` and `EX`).

## Review 

In our first Hello World program, we introduced the GET and SET operations using the
familiar key-value paradigm, the string datatype provides additional functionality and
flexibility beyond the basic GET and SET operations.

In this chapter, we used the string data type to store vote counts for blog posts on an
website. We saw how strings could be treated as integers, allowing us to update vote
counts directly in Redis using increment and decrement commands. We also saw how strings
could be treated as bitmaps to provide compact representations of boolean information.
Finally, we learned about several of the variants of GET and SET which support multiple
keys, manage time-to-live values, and perform test-and-set operations.

Redis strings are not like the strings we think of from programming languages. Redis
strings can be used to store many different kinds of data, data we don't normally think of
as string data. We've seen how the GET and SET operations are used to implement a basic
key-value store and we've also seen how other operations can be used to manipulate strings
as integers, floating points and bitmaps.

This chapter has covered a wide range of different string functionality, but not all of
the commands are covered in this chapter. For additional information on Redis strings as
well as a complete command reference, see 
[String Commands](https://redis.io/commands#string) page at [Redis.io](http://www.redis.io).